<a href="https://colab.research.google.com/github/Daniel-Oduntan/FDSfE_DOduntan/blob/main/HW7/Copy_of_Daniel's_NLP_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis of Movie Reviews:
## Data - 
the data comes from the Kaggla challenge https://www.kaggle.com/code/yagli18/sentiment-analysis-on-movie-reviews and is comprosed of ~10k movie reviews and associated ratings. 
The challenge there is to create a model that predicts the rating based on the review, which is a Natural Language Processing sentiment analysis task. Sentiment analysis measures the "sentiment" (typically in a positive-to-negative linear scale) of a text. There are many issues with sentiment analysis in genera, and in particulat here, the "sentiment" of the review (negative, which is associated with sad and angry for example) may not match with the judgement on the quality of the movie (which may be sad, but good if it is a drama) 

**We will simply apply a pre-trained model that weights each word in the text to measure its negative-to-positive sentiment and compounds all the words to get an overall sentiment. The right way to do this tho would be to _train_ a new model on the data, learning the match between the sentiment of the words and the rating assigned by the author of the review. That way, the model is _specific_ to this data. This is a challenge that I live for you!**

In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=885efc36ba6715b2d14cd9d7adae0cdc62c41187a3867e2a00077a205a44bf87
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [3]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 4.5 MB/s 


In [4]:
import os
import json
import pandas as pd

import numpy as np
import pylab as plt
import seaborn as sns

import nltk
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import statistics as st

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [7]:
env = json.load(open("kaggle.json", "r"))
os.environ["KAGGLE_USERNAME"] = env["username"]
os.environ["KAGGLE_KEY"] = env["key"]

In [8]:
!kaggle datasets sentiment

usage: kaggle datasets [-h]
                       {list,files,download,create,version,init,metadata,status}
                       ...
kaggle datasets: error: argument command: invalid choice: 'sentiment' (choose from 'list', 'files', 'download', 'create', 'version', 'init', 'metadata', 'status')


In [9]:
!mkdir sentimentanalysis

mkdir: cannot create directory ‘sentimentanalysis’: File exists


In [10]:
cd sentimentanalysis/

/content/drive/MyDrive/sentimentanalysis


In [11]:
!kaggle competitions download -c sentiment-analysis-on-movie-reviews

sentiment-analysis-on-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [12]:
!ls *

sampleSubmission.csv			 test.tsv.zip
sentiment-analysis-on-movie-reviews.zip  train.tsv.zip


In [ ]:
!unzip  sentiment-analysis-on-movie-reviews.zip


Archive:  sentiment-analysis-on-movie-reviews.zip
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
reviews = pd.read_csv("train.tsv.zip", sep="\t")
reviews

In [ ]:
reviews = reviews.groupby("SentenceId").first()
reviews

In [ ]:
reviews.shape

In [ ]:
reviews[["Phrase", "Sentiment"]].values

In [ ]:
print('Number of Reviews/Documents: {}'.format(len(reviews)))

print('Corpus Size (words): {}'.format(np.sum([len(phrase.split()) for phrase in reviews["Phrase"].values])))

In [ ]:
reviews.Sentiment.unique()

In [ ]:
#plot sentiment histogram
plt.hist(reviews["Sentiment"], bins=[-0.5, 0.5, 1.5, 2.5, 3.5, 4.5]);

In [ ]:
!python -m nltk.downloader punkt

In [ ]:
#NLP packs
#import nltk  --> used in this notebook
#from nltk.tokenize import word_tokenize --> used in this notebook
#from nltk.corpus import stopwords
#from stop_words import get_stop_words --> used in this notebook
#from textblob import TextBlob , Word
#import re 
#import string

In [ ]:
#remove some characters like new line from the strings
reviews['Phrase'] = reviews['Phrase'].str.lower(
    ).str.replace(
        '[^\w\s]', '').str.replace(
            '\d+', '').str.replace(
                '\n',' ').replace(
                        '\r','').str.replace(
                            "[^a-zA-Z0-9\s]",'') 

# lets talk about the lambda construct and apply

In [ ]:
reviews['Phrase'] = reviews['Phrase'].apply(lambda x: word_tokenize(x)) # sentence -> words
reviews['Phrase']

# lets talk about stop words... and list comprehensions

In [ ]:
STOPWORDS

In [ ]:
reviews["Phrase"] = reviews["Phrase"].apply(lambda x: [word for word in x if word not in STOPWORDS]) # stop words removal


# Frequency of words

In [ ]:
nltk.FreqDist(reviews.Phrase.sum()).most_common(10)

In [ ]:
#reviews["Phrase"] = reviews["Phrase"].apply(lambda x : 
#                  [word for word in x if word not in 
#                   ["s", "nt", "rrb", "lrb", "one"]]) # more "stop" words 
# or I could do it by length: are there gpoing to be key words that are <4 characters??
reviews["Phrase"] = reviews["Phrase"].apply(lambda x : 
                  [word for word in x if len(word)>3]) # more "stop" words 


In [ ]:
words_freq = pd.DataFrame(nltk.FreqDist(reviews.Phrase.sum(
                                          )).most_common(10), 
                          columns=['Top Words', 'Frequency'])
ax = sns.barplot(x="Top Words",y="Frequency",data=words_freq)
plt.xticks(rotation = 45); # Rotates X-Axis Ticks by 45-degrees


# Lets talk about the str.join() method

In [ ]:
def wc(data,bgcolor): # word cloud
    plt.figure(figsize=(10,10))
    wc = WordCloud(background_color=bgcolor, max_words=100, 
                 min_word_length=4, contour_width=3, 
                 contour_color='steelblue')
    wc.generate(" ".join(data)) # tum kelimeleri birlestirip tek 1 string haline getirir
    plt.imshow(wc)
    plt.axis("off")


#TASK 1 : 
Use the function above to plot the word cloud **for Sentiment==0**, comment on the plot to describe how a wordcloud should be read and what are interesting words that appear

In [ ]:
# your code here
#plot the word cloud for sentiment 0
wc(reviews[reviews["Sentiment"]==0]["Phrase"].sum(), bgcolor="white")

This figure utilizes a function to look at the first 100 words that are in the Phrase column with a sentiment of zero and creates a word cloud that makes the size of the word proportional to how frequently the word appears. Some interesting words include "stupid", "worst", "dull", and "predictable".

#TASK 2 : 
Use the function above to plot the word cloud **for Sentiment==4**, comment on the plot to describe how a wordcloud should be read and what are interesting words that appear. How does it differ from Sentiment==0?

In [ ]:
# your code here
#plot the word cloud for sentiment 4
wc(reviews[reviews["Sentiment"]==4]["Phrase"].sum(), bgcolor="white")

This figure utilizes a function to look at the first 100 words that are in the Phrase column with a sentiment of four and creates a word cloud that makes the size of the word proportional to how frequently the word appears. Some interesting words include "funny", "love", "comedy", and "entertaining".

# Task 3: sentiment analysis

install a package that contains a pretrained model: this model will score every word in a sentence as positive or negative (with a continuous score from -1 to 1) and will then average over those words.

In [ ]:
#create a model
sid_obj = SentimentIntensityAnalyzer() 
#apply the model to a string
sid_obj.polarity_scores("series escapades demonstrating adage good goose good gander occasionally amuses none amounts much story")
 

In [ ]:
#define a function to generate the senriment given a string
def sentiment_vader(words):
  sid_obj = SentimentIntensityAnalyzer()
  sentiment_dict = sid_obj.polarity_scores(' '.join(words))
  return sentiment_dict['compound']

## TASK steps: 
1. take the review for each Sentiment column value (you can use Sentiment==0, Sentiment==1..... like you should have done for the wordclouds) 
2. for each Sentiment group calculate the sentiment of each Phrase, the mean sentiment across the Phrases, and the standard deviation over this mean
3. plot on the x axis the value of the Sentiment column [0, 1, 2, 3, 4]
4. plot on the y axis the _mean_ sentiment as measured by your code for all reviews with that Sentiment value in the dataset
5. include the errorbar on the measured sentiment (use plt.errorbar to do this)
6. Comment on the figure: what do you see? is the model good?

### hint: you may want to use a list comprehension to do this! the steps are not necessarily sequential: everything can be done with a single line of code. This is not required however

In [ ]:
s0 = reviews[reviews["Sentiment"]==0]["Phrase"]
s0

In [ ]:
s1 = reviews[reviews["Sentiment"]==1]["Phrase"]
s1

In [ ]:
s2 = reviews[reviews["Sentiment"]==2]["Phrase"]
s2

In [ ]:
s3 = reviews[reviews["Sentiment"]==3]["Phrase"]
s3

In [ ]:
s4 = reviews[reviews["Sentiment"]==4]["Phrase"]
s4

In [ ]:
s0mean = st.mean(sentiment_vader(Phrase) for Phrase in s0.values)
print("The mean of phrases with 0 sentiment = {0:.3f}".format(s0mean))

In [ ]:
s0std = st.stdev(sentiment_vader(Phrase) for Phrase in s0.values)
print("The standard deviation of phrases with 0 sentiment = {0:.3f}".format(s0std))

In [ ]:
s1mean = st.mean(sentiment_vader(Phrase) for Phrase in s1.values)
print("The mean of phrases with 1 sentiment = {0:.3f}".format(s1mean))

In [ ]:
s1std = st.stdev(sentiment_vader(Phrase) for Phrase in s1.values)
print("The standard deviation of phrases with 1 sentiment = {0:.3f}".format(s1std))

In [ ]:
s2mean = st.mean(sentiment_vader(Phrase) for Phrase in s2.values)
print("The mean of phrases with 2 sentiment = {0:.3f}".format(s2mean))

In [ ]:
s2std = st.stdev(sentiment_vader(Phrase) for Phrase in s2.values)
print("The standard deviation of phrases with 2 sentiment = {0:.3f}".format(s2std))

In [ ]:
s3mean = st.mean(sentiment_vader(Phrase) for Phrase in s3.values)
print("The mean of phrases with 3 sentiment = {0:.3f}".format(s3mean))

In [ ]:
s3std = st.stdev(sentiment_vader(Phrase) for Phrase in s3.values)
print("The standard deviation of phrases with 3 sentiment = {0:.3f}".format(s3std))

In [ ]:
s4mean = st.mean(sentiment_vader(Phrase) for Phrase in s4.values)
print("The mean of phrases with 4 sentiment = {0:.3f}".format(s4mean))

In [ ]:
s4std = st.stdev(sentiment_vader(Phrase) for Phrase in s4.values)
print("The standard deviation of phrases with 4 sentiment = {0:.3f}".format(s4std))

In [ ]:
x = [0, 1, 2, 3, 4]
y = [s0mean, s1mean, s2mean, s3mean, s4mean]
plt.errorbar(x, y, yerr=[s0std, s1std, s2std, s3std, s4std], ecolor="Red", fmt="o")
plt.xlabel("Sentiments")
plt.ylabel("Mean of Sentiments");

The figure above shows the error bar for sentiment predictions with the sentiment vader package.The significant difference between the data point and the cap length shows that the vadar package is inappropriate for this dataset